In [54]:
import pandas as pd
from pathlib import Path

# Load the wide dataset
path = Path(r"C:\Users\kfq6\Documents\Data\LABKA_wide_rawSvar.xlsx")
df = pd.read_excel(path)

path = Path(r"C:\Users\kfq6\Documents\Data\LABKA_wide_rawSvar.xlsx")
df = pd.read_excel(path, dtype=str)  # read everything as string first

# Decode Excel-style XML escapes like _x0038_ → 8, _x002C_ → ,
def decode_excel_xml(s):
    if not isinstance(s, str):
        return s
    return re.sub(r"_x([0-9A-Fa-f]{4})_", lambda m: chr(int(m.group(1), 16)), s)

df = df.applymap(decode_excel_xml)


C:\Users\kfq6\AppData\Local\Temp\37\ipykernel_80720\344398136.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(decode_excel_xml)


In [55]:
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57480 entries, 0 to 57479
Data columns (total 22 columns):
 #   Column                                                                                                       Non-Null Count  Dtype 
---  ------                                                                                                       --------------  ----- 
 0   DW_EK_Borger                                                                                                 57480 non-null  object
 1   Testdato                                                                                                     57480 non-null  object
 2   Alder_Proevetagningstid                                                                                      57480 non-null  object
 3   Dato_Proevetagningstid                                                                                       57480 non-null  object
 4   Klok_Proevetagningstid                                          

In [56]:
df.head()

,DW_EK_Borger,Testdato,Alder_Proevetagningstid,Dato_Proevetagningstid,Klok_Proevetagningstid,B-Hæmoglobin_[mmol_x002F_l],Hb_x0028_B_x0029_-Hæmoglobin_x0020_A1c_x0020__x0028_IFCC_x0029__[mmol_x002F_mol],P-25-Hydroxy-Vitamin_x0020_D_x0028_D3_x002B_D2_x0029__[nmol_x002F_l],P-Albumin_[g_x002F_l],P-Calcium_[mmol_x002F_l],...,P-Kolesterol_[mmol_x002F_l],P-Kolesterol_x0020_HDL_[mmol_x002F_l],P-Kolesterol_x0020_LDL_[mmol_x002F_l],P-Kreatinin_[_x00B5_mol_x002F_l],P-Natrium_[mmol_x002F_l],P-Triglycerid_[mmol_x002F_l],P-Vitamin_x0020_B12_[pmol_x002F_l],U-Albumin_x0020__x002F__x0020_Kreatinin-ratio_[_x00D7__x0020_10_x003C_sup_x003E_-3_x003C__x002F_sup_x003E_],U-Albumin_x0020__x002F__x0020_Kreatinin-ratio_[mg_x002F_g],eGFR_x0020__x002F__x0020_1_x002C_73m_x00B2__x0028_CKD-EPI_x0029__[ml_x002F_min]
0,2611,2022-02-10 00:00:00,56,2022-02-10 00:00:00,1960-01-01 07:25:00,"8,5",51,NaN,35,"2,14",...,NaN,NaN,NaN,164,141,NaN,NaN,"803,7",NaN,40
1,2611,2022-06-02 00:00:00,56,2022-06-02 00:00:00,1960-01-01 07:19:00,"8,8",74,NaN,39,"2,34",...,"5,6","0,92","Ikke beregnet, da Tgly > 4,5 mmol/l",148,138,"4,7",NaN,"1653,2",NaN,45
2,2611,2022-06-09 00:00:00,56,2022-06-09 00:00:00,1960-01-01 11:57:00,"9,2",NaN,NaN,44,"2,46",...,NaN,NaN,NaN,149,139,NaN,NaN,NaN,NaN,45
3,2611,2022-08-18 00:00:00,56,2022-08-18 00:00:00,1960-01-01 07:31:00,"8,6",111,NaN,38,"2,29",...,"5,7","0,86","Ikke beregnet, da Tgly > 4,5 mmol/l",147,138,"5,0",NaN,"1843,8",NaN,45
4,2611,2022-09-21 00:00:00,56,2022-09-21 00:00:00,1960-01-01 07:17:00,NaN,112,NaN,NaN,NaN,...,"5,4","0,93","2,8",151,138,"3,6",164,NaN,2447,44


In [57]:
# Columns that should stay as-is
# -----------------------------
meta_cols = ["DW_EK_Borger", "Testdato", "Alder_Proevetagningstid",
             "Dato_Proevetagningstid", "Klok_Proevetagningstid"]

# -----------------------------
# Fix decimal commas and coerce to numeric for all non-meta columns
# -----------------------------
num_df = df.copy()

for c in num_df.columns:
    if c not in meta_cols:
        # Replace comma decimals with dot
        num_df[c] = num_df[c].astype(str).str.replace(",", ".", regex=False)
        # Optional: strip whitespace
        num_df[c] = num_df[c].str.strip()
        # Convert to numeric (anything non-numeric becomes NaN)
        num_df[c] = pd.to_numeric(num_df[c], errors='coerce')

print("\nTypes after numeric coercion:")
print(num_df.info())

# -----------------------------
# Date parsing + basic cleaning
# -----------------------------
ID_COL = "DW_EK_Borger"
DATE_COL = "Testdato"

num_df[DATE_COL] = pd.to_datetime(num_df[DATE_COL], errors="coerce")
num_df = num_df.dropna(subset=[ID_COL, DATE_COL]).sort_values([ID_COL, DATE_COL]).reset_index(drop=True)

# Only keep numeric lab columns (exclude meta/date)
lab_cols = [c for c in num_df.columns
            if c not in meta_cols and pd.api.types.is_numeric_dtype(num_df[c])]

if not lab_cols:
    raise ValueError("No numeric lab columns detected after coercion. Check your input and parsing.")



Types after numeric coercion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57480 entries, 0 to 57479
Data columns (total 22 columns):
 #   Column                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                       --------------  -----  
 0   DW_EK_Borger                                                                                                 57480 non-null  object 
 1   Testdato                                                                                                     57480 non-null  object 
 2   Alder_Proevetagningstid                                                                                      57480 non-null  object 
 3   Dato_Proevetagningstid                                                                                       57480 non-null  object 
 4   Klok_Proevetagningstid     

In [58]:
num_df.head()

,DW_EK_Borger,Testdato,Alder_Proevetagningstid,Dato_Proevetagningstid,Klok_Proevetagningstid,B-Hæmoglobin_[mmol_x002F_l],Hb_x0028_B_x0029_-Hæmoglobin_x0020_A1c_x0020__x0028_IFCC_x0029__[mmol_x002F_mol],P-25-Hydroxy-Vitamin_x0020_D_x0028_D3_x002B_D2_x0029__[nmol_x002F_l],P-Albumin_[g_x002F_l],P-Calcium_[mmol_x002F_l],...,P-Kolesterol_[mmol_x002F_l],P-Kolesterol_x0020_HDL_[mmol_x002F_l],P-Kolesterol_x0020_LDL_[mmol_x002F_l],P-Kreatinin_[_x00B5_mol_x002F_l],P-Natrium_[mmol_x002F_l],P-Triglycerid_[mmol_x002F_l],P-Vitamin_x0020_B12_[pmol_x002F_l],U-Albumin_x0020__x002F__x0020_Kreatinin-ratio_[_x00D7__x0020_10_x003C_sup_x003E_-3_x003C__x002F_sup_x003E_],U-Albumin_x0020__x002F__x0020_Kreatinin-ratio_[mg_x002F_g],eGFR_x0020__x002F__x0020_1_x002C_73m_x00B2__x0028_CKD-EPI_x0029__[ml_x002F_min]
0,1000342,2022-06-15,38,2022-06-15 00:00:00,1960-01-01 09:02:00,NaN,53.0,53.0,NaN,NaN,...,4.6,1.4,2.9,73.0,140.0,0.8,446.0,NaN,NaN,NaN
1,1000342,2023-03-29,39,2023-03-29 00:00:00,1960-01-01 08:20:00,NaN,55.0,NaN,NaN,NaN,...,4.2,1.3,2.4,73.0,138.0,1.1,408.0,NaN,NaN,NaN
2,1000342,2024-06-05,40,2024-06-05 00:00:00,1960-01-01 08:33:00,9.8,62.0,NaN,NaN,NaN,...,4.4,1.4,2.5,75.0,140.0,1.0,446.0,NaN,NaN,NaN
3,1000342,2024-10-11,41,2024-10-11 00:00:00,1960-01-01 13:57:00,9.7,63.0,62.0,NaN,NaN,...,4.5,1.5,2.5,102.0,142.0,1.1,531.0,NaN,108.0,78.0
4,1000342,2025-10-10,42,2025-10-10 00:00:00,1960-01-01 10:39:00,9.5,50.0,NaN,NaN,NaN,...,4.0,1.4,2.2,82.0,142.0,0.8,497.0,NaN,NaN,NaN


In [59]:
# -----------------------------
# Build 3-month windows per patient, anchored at their LAST test
# -----------------------------
last_date = num_df.groupby(ID_COL)[DATE_COL].transform("max")

# Months difference from each row to the patient-specific last date
months_diff = (last_date.dt.year - num_df[DATE_COL].dt.year) * 12 + (last_date.dt.month - num_df[DATE_COL].dt.month)
num_df["window_idx"] = (months_diff // 3).astype("int64")

# Window bounds per row (consistent within each patient-window)
num_df["_last_date"] = last_date
num_df["window_end"] = num_df.apply(lambda r: r["_last_date"] - pd.DateOffset(months=3 * int(r["window_idx"])), axis=1)
num_df["window_start"] = num_df["window_end"] - pd.DateOffset(months=3) + pd.Timedelta(days=1)

group_cols = [ID_COL, "window_idx", "window_start", "window_end"]

# -----------------------------
# Aggregate: per patient × window means
# -----------------------------
agg_dict = {DATE_COL: ["min", "max", "count"]}
agg_dict.update({c: "mean" for c in lab_cols})

out = (
    num_df.groupby(group_cols, dropna=False)
          .agg(agg_dict)
          .reset_index()
)

# Flatten multiindex columns from aggregation
out.columns = ["_".join([c for c in col if c]) if isinstance(col, tuple) else col for col in out.columns]
out = out.rename(columns={
    f"{DATE_COL}_min": "first_sample_in_window",
    f"{DATE_COL}_max": "last_sample_in_window",
    f"{DATE_COL}_count": "n_samples_in_window",
})

out = out.sort_values([ID_COL, "window_idx"]).reset_index(drop=True)

In [60]:
# -----------------------------
# Sanity checks (because trust is earned)
# -----------------------------
assert out["window_idx"].min() >= 0
assert (out["last_sample_in_window"] >= out["first_sample_in_window"]).all()

# Quick peek at the means
print("\nPreview of per-window means:")
print(out[[ID_COL, "window_idx"] + [c for c in out.columns if c.endswith("_mean")][:6]].head(12))


Preview of per-window means:
   DW_EK_Borger  window_idx  B-Hæmoglobin_[mmol_x002F_l]_mean  \
0       1000342           0                          9.500000   
1       1000342           4                          9.700000   
2       1000342           5                          9.800000   
3       1000342          10                               NaN   
4       1000342          13                               NaN   
5       1000374           0                          6.850000   
6       1000374           1                          6.400000   
7       1000374           2                          6.133333   
8       1000374           3                          6.400000   
9       1000374           4                          6.350000   
10      1000374           5                          6.466667   
11      1000374           6                          6.800000   

    Hb_x0028_B_x0029_-Hæmoglobin_x0020_A1c_x0020__x0028_IFCC_x0029__[mmol_x002F_mol]_mean  \
0                              

In [62]:
# -----------------------------
# Round the means so they stop screaming
# -----------------------------
mean_cols = [c for c in out.columns if c not in group_cols + ["first_sample_in_window", "last_sample_in_window", "n_samples_in_window"]]
# If you want uniform rounding everywhere:
out[mean_cols] = out[mean_cols].round(3)

# If you want per-analyte precision, do a dict like:
# round_map = {
#     "Hb(B)-Hæmoglobin A1c (IFCC)_mean": 1,
#     "eGFR / 1,73m²(CKD-EPI)_mean": 0,
#     ...
# }
# for col, nd in round_map.items():
#     if col in out.columns:
#         out[col] = out[col].round(nd)


In [64]:
csv_path = Path(r"C:\Users\kfq6\Documents\Data\labs_3m_windows_avg.csv")
out.to_csv(csv_path, index=False, sep=';', encoding='utf-8-sig')
print(f"\nSaved: {csv_path}  |  Rows: {len(out):,}  |  Cols: {len(out.columns)}")


Saved: C:\Users\kfq6\Documents\Data\labs_3m_windows_avg.csv  |  Rows: 24,209  |  Cols: 24
